In [2]:
import pandas as pd
import sys

sys.path.append('../')

In [3]:
import helper

In [4]:
labelfile = '../Data/ThioesteraseProductFractionLabel.xlsx'
seqfile = '../Data/Enzyme_Dataset.txt'

In [5]:
df = pd.read_excel(labelfile,header=None)

In [6]:
Xnameref,y = df.iloc[:,0].values,df.iloc[:,1].values

In [7]:
seq,label,Xname = helper.parseEnzymeFile(seqfile)

In [8]:
seqdictmap = dict(zip(Xname,seq))

In [9]:
labeldictmap = dict(zip(Xnameref,y))

In [10]:
enzname = []
X = []
y = []

for xn in seqdictmap.keys():
    if xn in labeldictmap:
        enzname.append(xn)
        X.append(seqdictmap[xn])
        y.append(labeldictmap[xn])
    else:
        print(xn)

Cuphea_viscosissima_(CvFatB1)
Cuphea_viscosissima_(CvFatB2)
Cuphea_carthagenesis_(CcFatB)


In [11]:
# with open('../Data/RegressionData.txt','w') as f:
#     for name,seq,label in zip(enzname,X,y):
#         f.write(name + ',' + seq + ',' + str(label))
#         f.write('\n')
    

In [12]:
len(enzname)

103

In [13]:
len(X)

103

In [14]:
import numpy as np
enzname,X,y = np.array(enzname),np.array(X),np.array(y)

In [15]:
from sklearn.svm import SVR

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
def get_model(n_comp):
#     steps = [('pca',PCA(n_components=n_comp)),('LR',LinearRegression())]
    steps = [('pca',PCA(n_components=n_comp)),('SVR',SVR())]
    pipe = Pipeline(steps)
    return pipe
    

In [18]:
from FeatEngg.ngramEnc import Ngram

In [19]:
teidx,tridx = helper.modified_split(enzname)

In [20]:
ngram = Ngram(X[tridx],7,1)
ngram.fit()

In [21]:
X_ng_tr = ngram.transform(X[tridx])

In [22]:
y_tr = y[tridx]

In [23]:
mypipe = get_model(40)

model = mypipe.fit(X_ng_tr,y_tr)

In [24]:
y_pred_tr = model.predict(X_ng_tr)

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_tr,y_pred_tr)

0.011166208871416484

In [27]:
X_ng_te = ngram.transform(X[teidx])

In [28]:
y_pred_te = model.predict(X_ng_te)

In [29]:
y_te = y[teidx]

In [30]:
mean_squared_error(y_te,y_pred_te)

0.032083198371674636

In [31]:
y_te

array([0.23      , 0.561     , 0.683     , 0.47272727, 0.083     ,
       0.109     , 0.81      , 0.486     , 0.35      , 0.403     ,
       0.799     , 0.        , 0.919     , 0.02160761, 0.343     ,
       0.52      , 0.087     , 0.001     , 0.167     , 0.00791766,
       0.044     , 0.98      , 0.388     , 0.477     ])

In [32]:
y_pred_te

array([0.27727397, 0.56409851, 0.60678747, 0.61904121, 0.32560692,
       0.46234146, 0.58022309, 0.26926058, 0.259993  , 0.56737253,
       0.73377411, 0.33459882, 0.68883832, 0.15224383, 0.50826801,
       0.42804332, 0.25099367, 0.1592824 , 0.21851386, 0.19532108,
       0.15068909, 0.84643282, 0.66252711, 0.55186996])

In [33]:
from FeatEngg.gaacEnc import GAAC

In [34]:
gaac = GAAC()
Xgaac = gaac.transform(X)

In [35]:
ngram = Ngram(Xgaac[tridx],7,1)
ngram.fit()

In [36]:
X_ngaac_tr = ngram.transform(Xgaac[tridx])

In [52]:
newpipe = get_model(40)

In [53]:
new_model = newpipe.fit(X_ngaac_tr,y_tr)

In [54]:
X_ngaac_te = ngram.transform(Xgaac[teidx])
y_pred_te = new_model.predict(X_ngaac_te)

In [55]:
mean_squared_error(y_te,y_pred_te)

0.02490700268813226

In [56]:
y_te

array([0.23      , 0.561     , 0.683     , 0.47272727, 0.083     ,
       0.109     , 0.81      , 0.486     , 0.35      , 0.403     ,
       0.799     , 0.        , 0.919     , 0.02160761, 0.343     ,
       0.52      , 0.087     , 0.001     , 0.167     , 0.00791766,
       0.044     , 0.98      , 0.388     , 0.477     ])

In [57]:
y_pred_te

array([0.18983587, 0.60409654, 0.61144767, 0.63326297, 0.25862359,
       0.43839968, 0.55704672, 0.28199395, 0.31248027, 0.56321048,
       0.7967865 , 0.22155424, 0.75827047, 0.21699955, 0.47409085,
       0.38170444, 0.15333932, 0.07815398, 0.15333932, 0.12441097,
       0.18342671, 0.86977744, 0.67073363, 0.5565467 ])

In [58]:
import pickle

file = pickle.load(open('../Data/TE_ML_Data_pickle.p','rb'))

In [59]:
pmap = dict(zip(file[0],file[1]))

In [60]:
Xp = np.array([pmap[e] for e in enzname])

In [62]:
pospipe = get_model(50)

posmodel = pospipe.fit(Xp[tridx],y[tridx])

In [63]:
y_pred_te = posmodel.predict(Xp[teidx])

In [64]:
mean_squared_error(y_te,y_pred_te)

0.037008174619685724

In [65]:
y_te

array([0.23      , 0.561     , 0.683     , 0.47272727, 0.083     ,
       0.109     , 0.81      , 0.486     , 0.35      , 0.403     ,
       0.799     , 0.        , 0.919     , 0.02160761, 0.343     ,
       0.52      , 0.087     , 0.001     , 0.167     , 0.00791766,
       0.044     , 0.98      , 0.388     , 0.477     ])

In [66]:
y_pred_te

array([0.18404035, 0.62628381, 0.62706767, 0.62780577, 0.26249361,
       0.76703471, 0.49748694, 0.33891463, 0.31828073, 0.64781553,
       0.7726876 , 0.10673095, 0.70219593, 0.16701442, 0.43603869,
       0.31722937, 0.21092143, 0.0600093 , 0.15200974, 0.16701442,
       0.18813523, 0.88351566, 0.40819417, 0.54262801])

In [67]:
enzname[teidx]

array(['Lindera_communis', 'Cuphea_viscosisssima_(CvB2MT31',
       'Cuphea_viscosisssima_(CvB2MT27', 'Cuphea_avigera_(Ca2FatB2)',
       'Diploknema_butyracea', 'rTE44', 'rTE15', 'rTE56',
       'Cuphea_viscosisssima_(CvB2MT28', 'Cuphea_viscosisssima_(CvB2MT36',
       'Cuphea_viscosisssima_(CvB2MT42', 'Helianthus_Annuus_(HaFatB1)',
       'Cuphea_viscosisssima_(CvB2MT40', 'Cuphea_hookeriana_(ChFatB1)',
       'Cuphea_viscosisssima_(CvB2MT2', 'Cocos_nucifera_(CnFatB3)',
       'Cuphea_viscosisssima_(CvB2MT43', 'Ricinus_communis',
       'Cuphea_viscosisssima_(CvFatB2', 'Cuphea_decandra_(Cd1FatB1)',
       'Cuphea_viscosisssima_(CvB2MT24', 'Cuphea_palustris_(CpFatB1)',
       'rTE8', 'Cuphea_viscosisssima_(CvB2MT37'], dtype='<U34')